# Hydroelastic contact between a compliant box and a rigid box
Damrong Guoy

From source installation of Drake, start this notebook by:
```
drake $ bazel run //tutorials:hydro_box_with_contact_results
```

This tutorial follows the pattern of [authoring_multibody_simulation.ipynb](./authoring_multibody_simulation.ipynb) by Zach Feng. If you are not familiar with Drake, you might want to study that one first.

# Introduction

This tutorial shows you how to set up simulations with hydroelastic contacts. It also shows you how to report contact results numerically and visualize them in MeshCat.

Originally this contact model was invented by Ryan Elandt as *Pressure Field Contact*. In Drake, we call our implmentation *Hydroelastic Contact*. The abstract of [Ryan Elandt's dissertation](https://ecommons.cornell.edu/handle/1813/112919) says:

> "*...This model is useful in situations where deformation in the contact region is qualitatively important, but need not be accurate. PFC is a model of, and not a convergent approximation of, continuum mechanics. In PFC, each of the two contacting objects is assigned an immutable internal virtual pressure' field. ... the PFC contact surface is defined as the set of points where the two pressure fields are equal. ...*"

If you are not familiar with hydroelastic contact, please read these two Medium articles:

- [Rethinking Contact Simulation for Robot Manipulation](https://medium.com/toyotaresearch/rethinking-contact-simulation-for-robot-manipulation-434a56b5ec88)
- [Drake: Model-based design in the age of robotics and machine learning.](https://medium.com/toyotaresearch/drake-model-based-design-in-the-age-of-robotics-and-machine-learning-59938c985515)

Hydroelastic contact is faster than Finite Element model but less accurate. Hydroelastic contact is slower than point contact (used in most simulators including Drake as the default) but exhibits more stable behavior in challenging situations involving rolling friction, energy dissipation, and non-convex geometries.

For simplicity, this tutorial shows contacts between a rigid hydroelastic geometry and a compliant hydroelastic geometry. Both of them are rigid in the sense of rigid body dynamics. They do not have mechanical deformation. In our implementation, a compliant hydroelastic geometry has a pressure field representation (on a tetrahedral mesh, for example), but a rigid hydroelastic geometry only has a boundary surface representation (a triangle surface mesh, for example). Their contact patch always lies on the surface of the rigid hydroelastic geometry.

We will have another tutorial to show contacts between two compliant hydroelastic geometries. Their contact patch will be the surface of points where their pressure fields are equal.

This tutorial uses boxes as geometric primitives. We will have another tutorial to use meshes for hydroelastic contacts.

## Outline

This tutorial will show you how to:

1. Define SDFormat string for a compliant hydroelastic box.

    * Define a visual geometry.
    * Define a collision geometry with hydroelastic properties.
    * Define mass and inertia properties.


2. Define SDFormat string for a rigid hydroelastic box.

3. Create Diagram of MultibodyPlant using the two boxes.

4. Visualize the simulation.

5. Create a leaf system to numerically report contact results at the end of simulation.

6. Add ContactVisualizer to visualize contact results.

## Start MeshCat

We will use MeshCat for visualization in another window. After `StartMeshcat()` below, move the MeshCat tab out as a separated window, so you can see both this notebook and the MeshCat window at the same time.

**Explain what is MeschCat**
**Record and playback**
**drake_visualizer is going away**
**Meshcat slidebar..more later**
**Third-party Meshcat**
**SSH? other computer**

![StartMeshcat](./images/StartMeshcat.png)

In [1]:
from pydrake.geometry import StartMeshcat

# Start the visualizer. The cell will output an HTTP link after the execution.
# Click the link and a MeshCat tab should appear in your browser.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7000


## Create compliant hydroelastic box

*Make sure you have the MeshCat tab opened in your browser; the link is shown immediately above.*

We will create and visualize a compliant hydroelastic box with SDFormat string.

We will use `ModelVisualizer` to verify the SDFormat description. It will tell MeshCat to show the box.

### Visual geometry

First we create a visual geometry of a box of size 10cm x 20cm x 40cm. Observe in MeshCat the X(red), Y(green), and Z(blue) axes. The `<size>0.10 0.20 0.40</size>` specifies the dimensions of the box in meters. The `<diffuse>` tag specifies the RGB and alpha values between 0 and 1.

![CompliantBox](./images/compliant_box.png)

In [2]:
from pydrake.visualization import ModelVisualizer

visual_box_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="CompliantBox">
    <pose>0 0 0 0 0 0</pose>
    <link name="compliant_box">
      <visual name="visual">
        <geometry>
          <box>
            <size>0.10 0.20 0.40</size>
          </box>
        </geometry>
        <material>
          <diffuse>1.0 1.0 1.0 0.5</diffuse>
        </material>
      </visual>
    </link>
  </model>
</sdf>
"""

# Visualize the SDFormat string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelsFromString(visual_box_sdf, "sdf")
visualizer.Run(loop_once=True)

<RunResult.STOPPED: 2>

### Collision geometry with hydroelastic properties

We will add the `<collision>` block with hydroelastic properties. Without `<collision>`, our object cannot make contact in the simulation.

We specify the same collision geometry (box of size 10x20x40 centimeters) as the visual geometry.

Drake also uses the term proximity for collision. This is the `<drake:proximity_properties>` block that control hydroelastic contacts:

        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>1e7</drake:hydroelastic_modulus>
          <drake:mu_dynamic>0.5</drake:mu_dynamic>
          <drake:hunt_crossley_dissipation>1.25</drake:hunt_crossley_dissipation>
          <drake:relaxation_time>0.1</drake:relaxation_time>
        </drake:proximity_properties>

Use `<drake:compliant_hydroelastic/>` to specify a compliant object.

We set `<drake:hydroelastic_modulus>` to 1e7 Pascals. It is comparable to high-density polyethylene (HDPE) with 1 GPa Young's modulus. Hydroelastic modulus is not a physical property but rather a numerical parameter to tune our contact model. As a rule of thumb, set hydroelastic modulus to about 1/100 of Young's modulus. 

We set `<drake:mu_dynamic>` (unitless) for the friction coefficient.

We set `<drake:hunt_crossley_dissipation>` to 1.25 seconds/meter as the dissipation constant for TAMSI contact solver, and we set `<drake:relaxation_time>` to 0.1 seconds as the dissipation constant for SAP contact solver. Here we set both of them, so we can switch between the two solvers. In the future, we expect SAP to support Hunt & Crossley dissipation model. See *Hydroelastic contact* in [MultibodyPlant](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_multibody_plant.html) documentation.

In [3]:
from pydrake.visualization import ModelVisualizer

collision_box_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="CompliantBox">
    <pose>0 0 0 0 0 0</pose>
    <link name="compliant_box">
      <collision name="collision">
        <geometry>
          <box>
            <size>0.10 0.20 0.40</size>
          </box>
        </geometry>
        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>1e7</drake:hydroelastic_modulus>
          <drake:mu_dynamic>0.5</drake:mu_dynamic>
          <drake:hunt_crossley_dissipation>1.25</drake:hunt_crossley_dissipation>
          <drake:relaxation_time>0.1</drake:relaxation_time>
        </drake:proximity_properties>
      </collision>
      <visual name="visual">
        <geometry>
          <box>
            <size>0.10 0.20 0.40</size>
          </box>
        </geometry>
        <material>
          <diffuse>1.0 1.0 1.0 0.5</diffuse>
        </material>
      </visual>
    </link>
  </model>
</sdf>
"""

# Visualize the SDFormat string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelsFromString(collision_box_sdf, "sdf")
visualizer.Run(loop_once=True)

<RunResult.STOPPED: 2>

### Mass property and inertia matrix

In order to simulate dynamics, we will add `<mass>` of 1kg and `<inertia>` matrix.

The `<inertia>` of the box follows this formula:

    Ixx = m(s.y² + s.z²)/12
    Iyy = m(s.x² + s.z²)/12
    Izz = m(s.x² + s.y²)/12
    
In practice, we can use `UnitInertia.SolidBox()` as shown below and copy the numbers into SDFormat string afterwards.

In [4]:
from pydrake.multibody.tree import UnitInertia

# box of size 0.1 x 0.2 x 0.4 meters
unit_inertia = UnitInertia.SolidBox(Lx=0.1, Ly=0.2, Lz=0.4)
# mass 1 kg
mass = 1

inertia_matrix3 = mass * unit_inertia.CopyToFullMatrix3()

print(inertia_matrix3)

[[0.01666667 0.         0.        ]
 [0.         0.01416667 0.        ]
 [0.         0.         0.00416667]]


In [5]:
from pydrake.visualization import ModelVisualizer

# Define a compliant hydroelastic box.
compliant_box_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="CompliantBox">
    <pose>0 0 0 0 0 0</pose>
    <link name="compliant_box">
      <inertial>
        <mass>1.0</mass>
        <inertia>
          <ixx>0.016666</ixx> <ixy>0.0</ixy> <ixz>0.0</ixz>
          <iyy>0.014166</iyy> <iyz>0.0</iyz>
          <izz>0.004166</izz>
        </inertia>
      </inertial>
      <collision name="collision">
        <geometry>
          <box>
            <size>0.10 0.20 0.40</size>
          </box>
        </geometry>
        <drake:proximity_properties>
          <drake:compliant_hydroelastic/>
          <drake:hydroelastic_modulus>1e7</drake:hydroelastic_modulus>
          <drake:mu_dynamic>0.5</drake:mu_dynamic>
          <drake:hunt_crossley_dissipation>1.25</drake:hunt_crossley_dissipation>
          <drake:relaxation_time>0.1</drake:relaxation_time>
        </drake:proximity_properties>
      </collision>
      <visual name="visual">
        <geometry>
          <box>
            <size>0.10 0.20 0.40</size>
          </box>
        </geometry>
        <material>
          <diffuse>1.0 1.0 1.0 0.5</diffuse>
        </material>
      </visual>
    </link>
  </model>
</sdf>
"""

# Visualize the SDFormat string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelsFromString(compliant_box_sdf, "sdf")
visualizer.Run(loop_once=True)

<RunResult.STOPPED: 2>

## Create rigid hydroelastic box

The following SDFormat string specifies a rigid hydroelastic box. It is similar to the compliant hydroelastic box.

Both the `<visual>` and `<collision>` geometries are a box of size 60cm x 1m x 5cm. Observe in MeshCat the X(red), Y(green), and Z(blue) axes.

![RigidBox](./images/rigid_box.png)


Use the tag `<drake:rigid_hydroelastic/>` to specify a rigid hydroelastic geometry. It does not use `<drake:hydroelastic_modulus>`. Conceptually it has infinite hydroelastic modulus.

This rigid hydroelastic box uses the same friction coefficient and dissipation constants as the previous compliant hydroelastic box.

We do not specify `<mass>` and `<inertia>` of the rigid box because, in the next section when we set up `Diagram`, we will fix the rigid box to the world frame. It will not move.
    
The `<frame name="top_surface">` is a frame at the top of the box with the tag:

    <pose relative_to="rigid_box_link">0 0 0.025 0 0 0</pose>

It is at 2.5cm (half of the box's height) above the center of the box. In the next section, we will create a scene that places the `top_surface` frame at the world's origin.

In [6]:
from pydrake.visualization import ModelVisualizer

# Create a rigid-hydroelastic table top
rigid_box_sdf = """<?xml version="1.0"?>
<sdf version="1.7">
  <model name="RigidBox">
    <link name="rigid_box_link">
      <visual name="visual">
        <pose>0 0 0 0 0 0</pose>
        <geometry>
          <box>
            <size>0.6 1.0 0.05</size>
          </box>
        </geometry>
        <material>
         <diffuse>0.9 0.8 0.7 0.5</diffuse>
        </material>
      </visual>
      <collision name="collision">
        <pose>0 0 0 0 0 0</pose>
        <geometry>
          <box>
            <size>0.6 1.0 0.05</size>
          </box>
        </geometry>
        <drake:proximity_properties>
          <drake:rigid_hydroelastic/>
          <drake:mu_dynamic>0.5</drake:mu_dynamic>
          <drake:hunt_crossley_dissipation>1.25</drake:hunt_crossley_dissipation>
          <drake:relaxation_time>0.1</drake:relaxation_time>
        </drake:proximity_properties>
      </collision>
    </link>
    <frame name="top_surface">
      <pose relative_to="rigid_box_link">0 0 0.025 0 0 0</pose>
    </frame>
  </model>
</sdf>

"""

# Visualize the SDFormat string you just defined.
visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.parser().AddModelsFromString(rigid_box_sdf, "sdf")
visualizer.Run(loop_once=True)

<RunResult.STOPPED: 2>

## Create Diagram of the scene

The function `create_scene()` below creates a scene using the two boxes that we created.

It uses `DiagramBuilder` to create `MultibodyPlant` and `SceneGraph`. It uses `Parser` to add the two SDFormat strings of the two boxes into `Diagram` of the scene.

It fixes the rigid box's top surface to the world frame by calling `WeldFrames()`.

It places the compliant box 1 meter above the rigid box.

After this step, the next section will add visualization to `DiagramBuilder`.

In [7]:
from pydrake.math import RigidTransform
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlant, MultibodyPlantConfig
from pydrake.systems.framework import DiagramBuilder

def create_scene(time_step=1e-3, solver="tamsi"):
    # Clear MeshCat window from the previous blocks.
    meshcat.Delete()
    meshcat.DeleteAddedControls()

    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlant(
        MultibodyPlantConfig(
            time_step=time_step,
            discrete_contact_solver=solver),
        builder)
    parser = Parser(plant)

    # Load the table top and the box we created.
    parser.AddModelsFromString(compliant_box_sdf, "sdf")
    parser.AddModelsFromString(rigid_box_sdf, "sdf")

    # Weld the rigid box to the world so that it's fixed during simulation.
    # The top surface passes the world's origin.
    plant.WeldFrames(plant.world_frame(), 
                     plant.GetFrameByName("top_surface"))

    # Finalize the plant after loading the scene.
    plant.Finalize()

    # Set how high the center of the compliant box is from the world's origin. 
    # W = the world's frame
    # C = frame at the center of the compliant box
    X_WC = RigidTransform(p=[0, 0, 1])
    plant.SetDefaultFreeBodyPose(plant.GetBodyByName("compliant_box"), X_WC)

    return builder, plant

## Set up visualization of the simulation

The function `create_scene()` above does not have visualization, so we will do it in the function `create_scene_and_viz()` below.

It uses `DiagramBuilder` from `create_scene()`. The `meshcat` interface represents the same MeshCat window that we have been using.

We set the `publish_period` to 1/256, so we can publish 256 frames per simulated second. In this example, we publish quite frequently, so later we can appreciate dynamics better.  See [VisualizationConfig](https://drake.mit.edu/doxygen_cxx/structdrake_1_1visualization_1_1_visualization_config.html) for more details.

At this step we set `publish_contacts` to `False`, we will set up contact visualization near the end of this tutorial.

We test the creation of `Diagram` by simulating for 0 second. It shows the compliant box 1 meter above the rigid box. You might have to zoom out to see both boxes.

![CompliantAboveRigidBox](./images/compliant_above_rigid_box.png)


In [8]:
from pydrake.visualization import ApplyVisualizationConfig, VisualizationConfig

def create_scene_and_viz(time_step=1e-3, solver="tamsi"):
    builder, plant = create_scene(time_step, solver)

    ApplyVisualizationConfig(
        config=VisualizationConfig(
                   publish_period = 1 / 256.0,
                   publish_contacts = False),
        builder=builder, meshcat=meshcat)
    
    return builder, plant


from pydrake.systems.analysis import Simulator

# Test creation of the diagram by simulating for 0 second.
# For now, use only the DiagramBuilder from the first return value and
# ignore the other two return values. We will use them later.
builder = create_scene_and_viz()[0]
simulator = Simulator(builder.Build())
simulator.Initialize()
simulator.AdvanceTo(0)

## Run the simulation

We define the function `run_simulation()` below. It builds `Diagram` using `DiagramBuilder` from the previous step. It passes `Diagram` to `Simulator`. It also records simulation result into `meshcat` for playback later. You should see the compliant box drops down to make contact with the rigid box.

In [9]:
from pydrake.systems.analysis import Simulator

def run_simulation(sim_time, time_step=1e-3, solver="tamsi"):
    builder, plant = create_scene_and_viz(time_step, solver)
    
    diagram = builder.Build()
    
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1.)
    
    meshcat.StartRecording(frames_per_second=256.0)
    simulator.AdvanceTo(sim_time)
    meshcat.StopRecording()


run_simulation(sim_time=10)

## Playback recording of the simulation

The following command `meshcat.PublishRecording()` will create a playback in MeshCat tab. It will enable `Animations` panel in MeshCat tab.

In [10]:
meshcat.PublishRecording()

In MeshCat window, click on `Open Controls` if you haven't done it. You should see `Animations` panel that look like this picture:

![Meshcat_play_pause_reset](./images/Meshcat_play_pause_reset.png)


Use the **play** button to replay the animation.

Adjust **timeScale** to control animation speed. Set it to 0.1 for slower playback to appreciate dynamics.

Use **pause** to stop animation in the middle, and use **reset** to move the animation back to time 0.


During the playback, you should see the compliant box drops down to make contact with the rigid box and bounces back at about 0.4s simulated time. The gravity will pull it down to make contact again. It will rock back and forth for a few seconds and reach steady state after about 6 seconds simulated time.

You can type **time** in the `Animations` panel to go to a particular snapshot in the animation. These pictures show some snapshots of the animation.

![CompliantRigidBoxAt0s](./images/compliant_rigid_box_00.0s.png)
![CompliantRigidBoxat0.4s](./images/compliant_rigid_box_00.4s.png)

# Report contact results numerically

We will show you how to report contact results. We will create a simple system to read contact results from `MultibodyPlant` and print them at the end of simulation.

## Use a simple system to publish contact results

If you are not familiar with creating a leaf system, please see the tutorial [Authoring Leaf Systems](./authoring_leaf_systems.ipynb).

We will define a leaf system called `ContactReporter` below. It has an input port that takes `ContactResults`. When there is a forced publish event, it will print the contact results from its input port. See `Publish(self, context)` in the code below. 

Contact results are available from an output port of `MultibodyPlant` by calling `get_contact_results_output_port()`. In `create_scene_with_contact_report()` below, we add `ContactReporter` to the diagram that we defined in the previous section. Then, we connect the contact results output port of `MultibodyPlant` to the input port of our `ContactReporter`. 

Our example has only one cotact patch between two geometries. However, the code below can report all contact patches from all pairs of geometries from hydroelastics.

See [ContactResults](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_contact_results.html) and [HydroelasticContactInfo](https://drake.mit.edu/doxygen_cxx/classdrake_1_1multibody_1_1_hydroelastic_contact_info.html) for more details.


**Periodic publish events***
**How to publish frequently**
**Witness event, when box contact***
**Report contact at my own frequency**

In [11]:
from pydrake.systems.framework import LeafSystem
from pydrake.common.value import AbstractValue
from pydrake.multibody.plant import ContactResults

class HydroelasticContactReporter(LeafSystem):
    def __init__(self):
        super().__init__()  # Don't forget to initialize the base class.
        self.DeclareAbstractInputPort(
            name="contact_results",
            model_value=AbstractValue.Make(
                # Input port will take ContactResults from MultibodyPlant
                ContactResults()))
        # Calling `ForcedPublish()` will trigger the callback.
        self.DeclareForcedPublishEvent(self.Publish)
        
    def Publish(self, context):
        print()
        print(f"ContactReporter::Publish() called at time={context.get_time()}")
        contact_results = self.get_input_port().Eval(context)
        
        num_hydroelastic_contacts = contact_results.num_hydroelastic_contacts()
        print(f"num_hydroelastic_contacts() = {num_hydroelastic_contacts}")
        
        for c in range(num_hydroelastic_contacts):
            print()
            print(f"hydroelastic_contact_info({c}): {c}-th hydroelastic contact patch")
            hydroelastic_contact_info = contact_results.hydroelastic_contact_info(c)
            
            spatial_force = hydroelastic_contact_info.F_Ac_W()
            print(f"F_Ac_W(): spatial force (on body A, at centroid of contact surface, in World frame) = \\")
            print(f"{spatial_force}")
                        
            print(f"contact_surface()")
            contact_surface = hydroelastic_contact_info.contact_surface()
            num_faces = contact_surface.num_faces()
            total_area = contact_surface.total_area()
            centroid = contact_surface.centroid()
            print(f"total_area(): area of contact surface in m^2 = {total_area}")
            print(f"num_faces(): number of polygons or triangles = {num_faces}")
            print(f"centroid(): centroid (in World frame) = {centroid}")        
        
        print()

def create_scene_with_contact_report(time_step=1e-3, solver="tamsi"):
    builder, plant = create_scene_and_viz(time_step, solver)
    
    contact_reporter = builder.AddSystem(HydroelasticContactReporter())    
    builder.Connect(plant.get_contact_results_output_port(),
                    contact_reporter.get_input_port(0))
        
    return builder, plant

## Run simulation and report contact results

As shown below, the function `run_simulation_with_contact_report()` calls `ForcedPublish()` after the simulation has finished.

The end of the code block calls `run_simulation_with_contact_report()` with `sim_time=0`. At time 0, the compliant box starts far above the rigid box. The printout will simply say there is no contact yet. It should look like this:

```
ContactReporter::Publish() called at time=0.0
num_hydroelastic_contacts() = 0
```

In [12]:
from pydrake.systems.analysis import Simulator

def run_simulation_with_contact_report(sim_time, time_step=1e-3, solver="tamsi"):
    builder, plant = create_scene_with_contact_report(time_step, solver)
    
    diagram = builder.Build()
    
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1.)
    
    meshcat.StartRecording(frames_per_second=256.0)
    simulator.AdvanceTo(sim_time)
    meshcat.StopRecording()

    # Forced publish after the simulation has finished.
    diagram.ForcedPublish(simulator.get_context())
    

run_simulation_with_contact_report(sim_time=0)
meshcat.PublishRecording()


ContactReporter::Publish() called at time=0.0
num_hydroelastic_contacts() = 0



We will run it again for 1 second simulated time. Since we will call `meshcat.PublishRecording()` on the second line, it will look like we drop the compliant box twice. The first animation is directly from the simulation, and the repeated animation is from the playback. You can zoom-in and slow down the playback in MeshCat tab (set `timeScale` to 0.1) to carefully observe dynamics.

After running the code below, you should see the spatial force `F_Ac_W` consisting of the torque `tau` and the force `f`. The force `f=[-1.4698008107796352, 1.2910859070622505, 9.190361581929913]` has the Z component about 9.19 newtons. It is less than 9.81 newtons, which is the gravity force acting on the compliant box with mass 1kg. This is because the compliant box is still rocking back and forth at the 1-second simulated time.

In [13]:
run_simulation_with_contact_report(sim_time=1)
meshcat.PublishRecording()


ContactReporter::Publish() called at time=1.0
num_hydroelastic_contacts() = 1

hydroelastic_contact_info(0): 0-th hydroelastic contact patch
F_Ac_W(): spatial force (on body A, at centroid of contact surface, in World frame) = \
SpatialForce(
  tau=[4.036986744178025e-07, 5.193892648279854e-07, 0.00011367920819944428],
  f=[-1.4698008107796352, 1.2910859070622505, 9.190361581929913],
)
contact_surface()
total_area(): area of contact surface in m^2 = 0.0014024481715244023
num_faces(): number of polygons or triangles = 4
centroid(): centroid (in World frame) = [ 0.0448088  -0.03969409  0.        ]



In the next code block, you will run the simulation for 10 seconds to steady state, and you should see `f.Z()` around 9.81 newtons equal to the gravity force:

>   f=[-2.0888656741505614e-05, 2.0072096007090746e-05, 9.809999999863855],

**World coordinates?**

In [14]:
run_simulation_with_contact_report(sim_time=10)
meshcat.PublishRecording()


ContactReporter::Publish() called at time=10.0
num_hydroelastic_contacts() = 1

hydroelastic_contact_info(0): 0-th hydroelastic contact patch
F_Ac_W(): spatial force (on body A, at centroid of contact surface, in World frame) = \
SpatialForce(
  tau=[-5.659221995885089e-06, -5.963533144012767e-06, 2.8882880887638432e-12],
  f=[-2.0888656741505614e-05, 2.0072096007090746e-05, 9.809999999863855],
)
contact_surface()
total_area(): area of contact surface in m^2 = 0.020000000000000004
num_faces(): number of polygons or triangles = 17
centroid(): centroid (in World frame) = [ 1.87675141e-06  1.32313266e-05 -8.32667268e-17]



# Visualize contact results

To visualize contact results, we will add `ContactVisualizer` to `Diagram` of the previous section.

The following function `create_scene_with_contact_report_and_viz()` adds `ContactVisualizer` to `DiagramBuilder` using `MultibodyPlant` and `meshcat`.

With `newtons_per_meter= 2e1`, it will draw a red arrow of length 1 meter for each force of 20 newtons. With `newtons_meters_per_meter= 1e-1`, it will draw a blue arrow of length 1 meter for each torque of 0.1 newton\*meters.

In [15]:
from pydrake.multibody.meshcat import ContactVisualizer, ContactVisualizerParams


def create_scene_with_contact_report_and_viz(time_step=1e-3, solver="tamsi"):
    builder, plant = create_scene_with_contact_report(time_step, solver)
    
    contact_viz = ContactVisualizer.AddToBuilder(
        builder, plant, meshcat,
        ContactVisualizerParams(
            publish_period= 1.0 / 256.0,
            newtons_per_meter= 2e1,
            newton_meters_per_meter= 1e-1))

    return builder, plant, contact_viz

## Run simulation with contact visualization

The following code will run the simulation and show results similar to the following picture. The red arrow represents the force `f`, and the blue arrow represents the torque `tau`. You should see the contact patch moving around together with the force and torque vectors. At the end of simulation, it will also report contact result numerically. 

![CompliantRigidBoxWithContactForceAt8s](./images/compliant_rigid_box_contact_force_8s.png)



In [16]:
from pydrake.systems.analysis import Simulator

def run_simulation_with_contact_report_and_viz(sim_time, time_step=1e-3, solver="tamsi"):
    builder, plant, contact_viz = create_scene_with_contact_report_and_viz(time_step, solver)
    
    diagram = builder.Build()
    
    simulator = Simulator(diagram)
    simulator.Initialize()
    simulator.set_target_realtime_rate(1)
    
    meshcat.StartRecording(frames_per_second=256.0)
    simulator.AdvanceTo(sim_time)
    meshcat.StopRecording()

    # Numerically report contact results at the end of simulation.
    diagram.ForcedPublish(simulator.get_context())

    
run_simulation_with_contact_report_and_viz(sim_time=6)


ContactReporter::Publish() called at time=6.0
num_hydroelastic_contacts() = 1

hydroelastic_contact_info(0): 0-th hydroelastic contact patch
F_Ac_W(): spatial force (on body A, at centroid of contact surface, in World frame) = \
SpatialForce(
  tau=[-0.0003021826009501072, 0.0008127015304672802, 1.6162165193418093e-13],
  f=[0.0029876260710093097, 0.0010618270221701494, 9.809999983916276],
)
contact_surface()
total_area(): area of contact surface in m^2 = 0.02000000000003324
num_faces(): number of polygons or triangles = 17
centroid(): centroid (in World frame) = [1.87739912e-06 1.32315594e-05 2.77555756e-17]



## Playback recording to appreciate dynamics

After running the code below, playback with `timeScale` = 0.1 or lower to appreciate dynamics. You should see the force and torque vectors oscillate synchronously with the rocking compliant box.



In [17]:
# In the current version, recording can show contact forces and torques well;
# however, contact surfaces are not recorded properly into MeshcatAnimation.
# For now, we delete contact surfaces to prevent confusion.
meshcat.Delete("/drake/contact_forces/hydroelastic/compliant_box+rigid_box_link/contact_surface");

meshcat.PublishRecording()

# Extra

In [18]:
# html_file = open("/home/damrongguoy/2023-04-04_record_hydro_box_with_contact_results.html", "wt")
# html_file.write(meshcat.StaticHtml())
# html_file.close()

In [19]:
# run_simulation_with_contact_report_and_viz(sim_time=0.405)
# meshcat.PublishRecording()

## Further reading

* [hydroelastic contact user guide](https://drake.mit.edu/doxygen_cxx/group__hydroelastic__user__guide.html)